In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import cv2
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from keras import layers
from keras.models import Sequential

2025-04-16 18:09:50.664454: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-16 18:09:51.040592: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-16 18:09:51.040678: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-16 18:09:51.116108: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-16 18:09:51.279009: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-16 18:09:51.281893: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
dataset_url = "http://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir=keras.utils.get_file('flower_photos',origin=dataset_url, cache_dir='.',untar=True)

In [3]:
import pathlib
data_dir=pathlib.Path(data_dir)

In [4]:
flowers_images_dict={
    'roses':list(data_dir.glob('roses/*.jpg')),
    'dandelion':list(data_dir.glob('dandelion/*.jpg')),
    'daisy':list(data_dir.glob('daisy/*.jpg')),
    'sunflowers':list(data_dir.glob('sunflowers/*.jpg')),
    'tulips':list(data_dir.glob('tulips/*.jpg')),
}

In [5]:
flowers_labels_dict={
    'roses':0,
    'dandelion':1,
    'daisy':2,
    'sunflowers':3,
    'tulips':4,
}

In [6]:
X,Y = [],[]
for flower_name,images in flowers_images_dict.items():
    for image in images:
        img =cv2.imread(str(image))
        resized_img=cv2.resize(img,(224,224))
        X.append(resized_img)
        Y.append(flowers_labels_dict[flower_name])

In [7]:
X=np.array(X)
Y=np.array(Y)

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=0)

In [9]:
# Replace X_train_scaled = X_train/255 with:
X_train_scaled = (X_train / 127.5) - 1.0  # Scale to [-1, 1]
X_test_scaled = (X_test / 127.5) - 1.0

In [10]:
feature_extractor='https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2'
pre_trained_model=hub.KerasLayer(feature_extractor, trainable=False)

In [11]:
data_augmentation = Sequential([
    layers.RandomFlip("horizontal",input_shape=(224, 224, 3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
    
])

In [11]:
num_classes=5
model = Sequential([
    layers.Input((224,224,3)),
    #data_augmentation, dosn't need it
    pre_trained_model,
    layers.Dense(num_classes, activation='softmax')  # Add softmax
])

In [12]:
model.compile(loss= 'sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [13]:
model.fit(X_train_scaled, Y_train, epochs=10, batch_size=4)

2025-04-16 18:14:21.943068: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1657012224 exceeds 10% of free system memory.


Epoch 1/10


2025-04-16 18:14:48.129186: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.


  3/688 [..............................] - ETA: 1:08 - loss: 2.3164 - accuracy: 0.0833     

2025-04-16 18:14:48.705382: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2025-04-16 18:14:48.793782: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2025-04-16 18:14:48.881679: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.


688/688 [==============================] - 60s 73ms/step - loss: 0.7016 - accuracy: 0.7449
Epoch 2/10
688/688 [==============================] - 51s 74ms/step - loss: 0.4015 - accuracy: 0.8550
Epoch 3/10
688/688 [==============================] - 50s 73ms/step - loss: 0.3054 - accuracy: 0.8935
Epoch 4/10
688/688 [==============================] - 50s 73ms/step - loss: 0.2486 - accuracy: 0.9179
Epoch 5/10
688/688 [==============================] - 50s 73ms/step - loss: 0.2016 - accuracy: 0.9310
Epoch 6/10
688/688 [==============================] - 50s 73ms/step - loss: 0.1600 - accuracy: 0.9542
Epoch 7/10
688/688 [==============================] - 50s 73ms/step - loss: 0.1384 - accuracy: 0.9622
Epoch 8/10
688/688 [==============================] - 50s 73ms/step - loss: 0.1097 - accuracy: 0.9738
Epoch 9/10
688/688 [==============================] - 51s 73ms/step - loss: 0.0996 - accuracy: 0.9753
Epoch 10/10
688/688 [==============================] - 50s 73ms/step - loss: 0.0806 - accurac

In [14]:
model.evaluate(X_test_scaled,Y_test)

29/29 [==============================] - 19s 632ms/step - loss: 0.5566 - accuracy: 0.8181


[0.5566027164459229, 0.8180828094482422]